In [1]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [2]:
from typing import Dict, Any
from langchain_core.messages import HumanMessage

from graph import create_debug_agent_graph

In [3]:
def fix_code(buggy_code: str, test_code: str = "", max_iterations: int = 5) -> Dict[str, Any]:
    """
    Fix buggy Python code using the AI agent.

    Args:
        buggy_code: The buggy Python code to fix
        test_code: Optional test code to validate the fix
        max_iterations: Maximum number of agent iterations

    Returns:
        Dictionary with fixed_code, success status, and iterations used
    """
    app = create_debug_agent_graph()

    # Prepare initial message
    user_message = f"""Fix the following buggy Python code:
```python
{buggy_code}
```
"""

    if test_code:
        user_message += f"""

Test code to validate the fix:
```python
{test_code}
```
"""

    # Run agent
    initial_state = {
        "messages": [HumanMessage(content=user_message)],
        "iterations": 0,
        "max_iterations": max_iterations,
        "original_buggy_code": buggy_code,
        "test_code": test_code,
        "fixed_code": "",
        "is_fixed": False
    }

    final_state = app.invoke(initial_state)

    return {
        "fixed_code": final_state.get("fixed_code", ""),
        "success": final_state.get("is_fixed", False),
        "iterations": final_state.get("iterations", 0),
        "messages": final_state["messages"]
    }


In [4]:
buggy_code = """
def calculate_average(numbers):
    total = 0
    for num in numbers:
        total += num
    return total / len(numbers)

# Test
print(calculate_average([1, 2, 3, 4, 5]))
print(calculate_average([]))  # This will cause an error!
"""

print("\nBuggy code:")
print(buggy_code)
print("\nRunning agent...\n")

result = fix_code(buggy_code, max_iterations=5)

print(f"Success: {result['success']}")
print(f"Iterations: {result['iterations']}")
print("\nFixed code:")
print(result['fixed_code'])


Buggy code:

def calculate_average(numbers):
    total = 0
    for num in numbers:
        total += num
    return total / len(numbers)

# Test
print(calculate_average([1, 2, 3, 4, 5]))
print(calculate_average([]))  # This will cause an error!


Running agent...

Success: True
Iterations: 2

Fixed code:
def calculate_average(numbers):
    if len(numbers) == 0:
        return None
    total = 0
    for num in numbers:
        total += num
    return total / len(numbers)

# Test
print(calculate_average([1, 2, 3, 4, 5]))
print(calculate_average([]))  # This will now return None
